# 具体应用：用Paddlepaddle做一个凡尔赛文学生成器

## 1. 安装相关库

In [1]:
# 检查paddlepaddle是否可以使用GPU
import paddle
paddle.utils.run_check()

Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


按照数据集要求对其进行整理，格式为“序号\t输入文本\t标签”

In [2]:
import pandas as pd

df = pd.read_excel("data.xlsx")
Keys = df["关键词"].values
Txts = df["文案"].values

with open("format_data.txt", "w",encoding='utf-8') as f:
    for i, k in enumerate(Keys):
        t = Txts[i]
        # t = "凡尔赛"
        f.write("{}\t{}\t{}\n".format(i, k, t))

## 2. 调用Paddlehub模型进行预训练

### 模型我们选用ERNIE-GEN模型

论文地址：https://arxiv.org/abs/2001.11314

![](https://ai-studio-static-online.cdn.bcebos.com/9698f8fef8004f579d01ca0d4a8a8255d58a8cc895c44d59b97141c82a23b833)


In [3]:
import paddlehub as hub

module = hub.Module(name="ernie_gen")

result = module.finetune(
    train_path='format_data.txt',
    save_dir="Versailles_param",
    max_steps=2000,
    noise_prob=0.1,
    save_interval=400,
    max_encode_len=60,
    max_decode_len=60
)

# 将训练参数打包为hub model
module.export(params_path=result['last_save_path'], module_name="Versailles", author="lyp")
!hub install Versailles

E:\anaconda3\envs\paddle_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E:\anaconda3\envs\paddle_env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-12-19 10:49:50,008] [    INFO] - Already cached C:\Users\11409\.paddlenlp\models\ernie-1.0\vocab.txt
[2023-12-19 10:49:50,022] [    INFO] - tokenizer config file saved in C:\Users\11409\.paddlenlp\models\ernie-1.0\tokenizer_config.json
[2023-12-19 10:49:50,023] [    INFO] - Special tokens file saved in C:\Users\11409\.paddlenlp\models\ernie-1.0\special_tokens_map.json
[2023-12-19 10:49:50,035] [    INFO] - Already cached C:\Users\11409\.paddlenlp\models\ernie-1.0\ernie_v1_chn_base.pdparams
[2023-12-19 10:49:50,037] [   DEBUG] - init ErnieModel with config

## 3. 运行预测

In [4]:
import paddlehub as hub

module = hub.Module(directory="Versailles")

[2023-12-19 11:02:07,560] [    INFO] - Already cached C:\Users\11409\.paddlenlp\models\ernie-1.0\ernie_v1_chn_base.pdparams
[2023-12-19 11:02:07,561] [   DEBUG] - init ErnieModel with config: {'attention_probs_dropout_prob': 0.1, 'hidden_act': 'relu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'max_position_embeddings': 513, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 18000, 'pad_token_id': 0}
[2023-12-19 11:02:07,795] [    INFO] - loading pretrained model from C:\Users\11409\.paddlenlp\models\ernie-1.0\ernie_v1_chn_base.pdparams
[2023-12-19 11:02:08,537] [    INFO] - param:mlm_bias not set in pretrained model, skip
[2023-12-19 11:02:08,539] [    INFO] - param:mlm.weight not set in pretrained model, skip
[2023-12-19 11:02:08,541] [    INFO] - param:mlm.bias not set in pretrained model, skip
[2023-12-19 11:02:08,542] [    INFO] - param:mlm_ln.weight not set in pretrained model, skip
[2023-12-19 11:02:08,543] [  

In [5]:
test_texts = ["凡尔赛"]
results = module.generate(texts=test_texts, use_gpu=False, beam_width=10)
for result in results[0]:
    print(result)

今天又是努力搬砖的一天根本没时间摸鱼中午只能休息两个小时，虽然五点就能准时下班，但早上十点就得到公司
我在沙发上写代码，突然牛奶热好了我去拿，回来的时候发现没保存。我问他：“我写到哪了？”他不慌不忙地说
今天又是努力搬砖的一天！根本没时间摸鱼，中午只能休息两个小时，虽然五点就能准时下班，但早上十点就得到
我在沙发上背单词，突然牛奶热好了我去拿，回来的时候发现没保存。我问他：“我背到哪了？”他不慌不忙地说
真羡慕那些可以随随便便离家出走的孩子，我都出来一个月了，还没走出我家草坪
今天又是努力搬砖的一天根本没时间摸鱼，中午只能休息两个小时，虽然五点就能准时下班，但早上十点就得到公
我在沙发上背单词，突然牛奶热好了我去拿，回来的时候发现没加书签。我问他：“我背到哪了？”他不慌不忙地
我在沙发上背单词，突然牛奶热好了我去拿，回来的时候发现没保存。我问他：“我写到哪了？”他不慌不忙地说
真羡慕那些可以随随便便离家出走的孩子，我都出来一个月了，还没走出我家草坪。
天天红烧肉，顿顿女儿红，这哪是人过的日子嘛


In [6]:
test_texts = ["晚餐"]
results = module.generate(texts=test_texts, use_gpu=False, beam_width=5)
for result in results[0]:
    print(result)

今天跟朋友在tiffanycafe聊天，想说一个词死活想不出来中文怎么说，顺口就把英文说出来了，朋友说：“你有必要吗？
今天又是努力搬砖的一天根本没时间摸鱼，中午只能休息两个小时，虽然五点就能准时下班，但早上十点就得到公
今天又是努力搬砖的一天根本没时间摸鱼中午只能休息两个小时，虽然五点就能准时下班，但早上十点就得到公司
今天和闺蜜逛街，柜姐还以为闺蜜是我经纪人。是因为我戴了gucci##20##52早春款墨镜吗？柜姐还说在电视上看见过我。
今天跟朋友在tiffanycafe聊天，想说一个词死活想不出来中文怎么说，顺口就把英文说出来了，朋友说：“我有必要吗？


In [7]:
test_texts = ["首饰"]
results = module.generate(texts=test_texts, use_gpu=False, beam_width=5)
for result in results[0]:
    print(result)

哎呀凡尔赛文学到底是什么啊我现在还没搞明白，昨天刚去山里度假没信号，都没跟上时代啦！我今天早上登微博
今天又是努力搬砖的一天根本没时间摸鱼，中午只能休息两个小时，虽然五点就能准时下班，但早上十点就得到公
唉，真羡慕年轻人啊，活力满满见个冠军奖杯那么激动，不像我，拿好几次都没啥感觉了，上次还差点手滑给摔了
哎呀凡尔赛文学到底是什么啊我现在还没搞明白，昨天刚去山里度假没信号，都没跟上时代啦！
真羡慕那些可以随随便便离家出走的孩子，我都出来一个月了，还没走出我家草坪


## 4. 改进方向：

可以看到数据集太少，出现了明显的过拟合现象，可以多搜集一些。